# Generating Music with Genetic Algorithms: From Melody Creation to MIDI and Music Sheet Visualization

### Description: In this notebook, we explore how Genetic Algorithms (GA) can be used to generate musical melodies. Starting with a basic scale and rhythmic structure, we evolved a melody through genetic processes, demonstrating the creative potential of AI. We then took the generated sequence and converted it into a MIDI file to hear the result, and visualized the melody in a music sheet format. This project showcases the intersection of AI and art, highlighting how algorithms can be used to generate original musical compositions.

### author: @jecs89
### Date: 14/11/2024
### Source: https://github.com/jecs89/LearningEveryDay

### Help by ChatGpt


# Genetic Algorithms
### Definition: https://en.wikipedia.org/wiki/Genetic_algorithm

### Representation
<img src='https://www.researchgate.net/publication/349241302/figure/fig3/AS:990383379070978@1613137206590/Genetic-selection-method-using-genetic-algorithm.png'>

In [5]:
import random

# Define the scales (using numbers representing the pitch classes)
# C major scale: C, D, E, F, G, A, B
# D minor scale: D, E, F, G, A, Bb, C
scales = {
    'C_major': ['C', 'D', 'E', 'F', 'G', 'A', 'B'],
    'D_minor': ['D', 'E', 'F', 'G', 'A', 'Bb', 'C']
}

# Genetic Algorithm settings
population_size = 10  # Number of melodies in each generation
melody_length = 20     # Number of notes in each melody
generations = 20      # Number of generations

# Define mutation rate
mutation_rate = 0.1

# Chromosome Representation: Each chromosome is a list of note indices (in scale)
def generate_melody(scale):
    """Generate a random melody within the given scale"""
    return [random.choice(scale) for _ in range(melody_length)]

def fitness(melody, scale):
    """Fitness function: check if all notes are within the scale"""
    return sum([1 for note in melody if note in scale])

def selection(population, fitness_values):
    """Select the top half of the population based on fitness"""
    sorted_population = [x for _, x in sorted(zip(fitness_values, population), reverse=True)]
    return sorted_population[:len(population)//2]

def crossover(parent1, parent2):
    """Single point crossover"""
    crossover_point = random.randint(1, len(parent1)-1)
    child1 = parent1[:crossover_point] + parent2[crossover_point:]
    child2 = parent2[:crossover_point] + parent1[crossover_point:]
    return child1, child2

def mutation(melody, scale):
    """Random mutation: change one note in the melody"""
    if random.random() < mutation_rate:
        mutation_point = random.randint(0, len(melody)-1)
        melody[mutation_point] = random.choice(scale)
    return melody

# Main GA loop
def genetic_algorithm(scale_name):
    scale = scales[scale_name]
    population = [generate_melody(scale) for _ in range(population_size)]

    for generation in range(generations):
        print(f"Generation {generation+1}")
        
        # Evaluate fitness of the population
        fitness_values = [fitness(melody, scale) for melody in population]
        best_fitness = max(fitness_values)
        print(f"Best fitness in this generation: {best_fitness}")
        
        # Selection: Select top half based on fitness
        selected_population = selection(population, fitness_values)
        
        # Crossover: Create new population
        new_population = []
        while len(new_population) < population_size:
            parent1, parent2 = random.sample(selected_population, 2)
            child1, child2 = crossover(parent1, parent2)
            new_population.extend([child1, child2])
        
        # Mutation: Apply mutation to the new population
        population = [mutation(melody, scale) for melody in new_population]

        # Print the best melody from the current generation
        best_melody = population[fitness_values.index(best_fitness)]
        print(f"Best melody: {best_melody}")
    
    # Return the best melody from the last generation
    fitness_values = [fitness(melody, scale) for melody in population]
    best_melody = population[fitness_values.index(max(fitness_values))]
    return best_melody

# Running the Genetic Algorithm for C Major Scale
best_melody = genetic_algorithm('C_major')
print(f"Best Melody: {best_melody}")

Generation 1
Best fitness in this generation: 20
Best melody: ['E', 'D', 'F', 'G', 'C', 'B', 'G', 'F', 'B', 'D', 'G', 'F', 'E', 'A', 'D', 'G', 'E', 'G', 'A', 'F']
Generation 2
Best fitness in this generation: 20
Best melody: ['E', 'D', 'F', 'G', 'F', 'A', 'C', 'G', 'G', 'D', 'G', 'F', 'E', 'A', 'D', 'G', 'E', 'G', 'A', 'F']
Generation 3
Best fitness in this generation: 20
Best melody: ['E', 'D', 'F', 'G', 'F', 'A', 'C', 'G', 'G', 'D', 'G', 'F', 'E', 'A', 'D', 'G', 'E', 'G', 'A', 'F']
Generation 4
Best fitness in this generation: 20
Best melody: ['E', 'D', 'F', 'G', 'F', 'A', 'D', 'G', 'G', 'D', 'G', 'F', 'E', 'A', 'D', 'G', 'E', 'G', 'A', 'F']
Generation 5
Best fitness in this generation: 20
Best melody: ['D', 'F', 'E', 'B', 'F', 'A', 'C', 'G', 'G', 'D', 'G', 'F', 'E', 'A', 'D', 'G', 'E', 'G', 'A', 'F']
Generation 6
Best fitness in this generation: 20
Best melody: ['E', 'F', 'F', 'B', 'F', 'A', 'C', 'G', 'G', 'D', 'G', 'F', 'E', 'A', 'D', 'G', 'E', 'G', 'A', 'F']
Generation 7
Best fitn

In [6]:
import mido
from mido import MidiFile, MidiTrack, Message

def melody_to_midi(melody, filename="generated_melody.mid"):
    """Convert a melody (list of note names) to a MIDI file."""
    
    # Map note names to MIDI note numbers
    note_to_midi = {
        'C': 60, 'D': 62, 'E': 64, 'F': 65, 'G': 67, 'A': 69, 'B': 71, 'Bb': 70
    }

    # Create a new MIDI file and track
    midi = MidiFile()
    track = MidiTrack()
    midi.tracks.append(track)
    
    # Add notes to the MIDI track
    for note in melody:
        midi_note = note_to_midi.get(note, 60)  # Default to C4 if note is unknown
        track.append(Message('note_on', note=midi_note, velocity=64, time=0))
        track.append(Message('note_off', note=midi_note, velocity=64, time=480))  # Duration of note

    # Save the MIDI file
    midi.save(filename)
    print(f"MIDI file saved as {filename}")

In [7]:
import pygame
import time

def play_midi(filename="generated_melody.mid"):
    """Play a MIDI file using pygame."""
    pygame.init()
    pygame.mixer.init()
    pygame.mixer.music.load(filename)
    pygame.mixer.music.play()

    # Wait for the music to finish playing
    while pygame.mixer.music.get_busy():
        time.sleep(0.5)

    pygame.mixer.quit()
    print("Playback finished")


pygame 2.6.1 (SDL 2.28.4, Python 3.9.13)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [9]:
# Assuming `best_melody` is the melody generated from the genetic algorithm
# best_melody = ['C', 'D', 'E', 'F', 'G', 'A', 'B', 'C']  # Example melody for testing

# Convert the melody to MIDI and play it
melody_to_midi(best_melody, "generated_melody.mid")
play_midi("generated_melody.mid")


MIDI file saved as generated_melody.mid
Playback finished


# Adding time duration for notes

To incorporate the duration of notes (e.g., whole notes or blancas, half notes or negras, eighth notes or corcheas), you can extend each note representation to include its duration. For example, each note could be a tuple (pitch, duration) where:

pitch represents the note (e.g., 'C', 'D', etc.).
duration represents its time value (e.g., 'whole', 'half', 'eighth').
Here’s a brief structure:

Durations (time values):

Blanca (whole): Longest duration (4 beats in 4/4 time).
Negra (half): Half the duration of a blanca (2 beats).
Corchea (eighth): Half the duration of a negra (1 beat).

In [10]:
import random

# Define possible rhythmic values (duration of each note in MIDI ticks)
rhythms = ['whole', 'half', 'quarter', 'eighth', 'sixteenth']
rhythm_durations = {
    'whole': 1920,    # 4 beats
    'half': 960,      # 2 beats
    'quarter': 480,   # 1 beat
    'eighth': 240,    # 0.5 beats
    'sixteenth': 120  # 0.25 beats
}

# Define the scale (C major scale for simplicity)
scales = {
    'C_major': ['C', 'D', 'E', 'F', 'G', 'A', 'B']
}

# Melody parameters
melody_length = 100  # Length of the melody (number of notes)
population_size = 100
generations = 50
mutation_rate = 0.2  # Mutation rate for note and rhythm

def generate_melody_with_rhythm(scale):
    """Generate a random melody with notes and rhythms"""
    melody = []
    for _ in range(melody_length):
        note = random.choice(scale)  # Randomly choose a note from the scale
        rhythm = random.choice(rhythms)  # Randomly choose a rhythm
        melody.append((note, rhythm))  # Add the (note, rhythm) pair to the melody
    return melody

def fitness_with_rhythm(melody, scale):
    """Fitness function: checks if all notes are within the scale"""
    return sum([1 for note, rhythm in melody if note in scale])  # Count valid notes

def crossover_with_rhythm(parent1, parent2):
    """Single point crossover for melody with rhythms"""
    crossover_point = random.randint(1, len(parent1) - 1)  # Random crossover point
    child1 = parent1[:crossover_point] + parent2[crossover_point:]
    child2 = parent2[:crossover_point] + parent1[crossover_point:]
    return child1, child2

def mutation_with_rhythm(melody, scale):
    """Random mutation: change one note or rhythm in the melody"""
    if random.random() < mutation_rate:
        mutation_point = random.randint(0, len(melody) - 1)  # Pick a random gene to mutate
        if random.random() < 0.5:
            # Change note
            melody[mutation_point] = (random.choice(scale), melody[mutation_point][1])
        else:
            # Change rhythm
            melody[mutation_point] = (melody[mutation_point][0], random.choice(rhythms))
    return melody

def selection(population, fitness_values):
    """Select the top half of the population based on fitness"""
    selected = [melody for _, melody in sorted(zip(fitness_values, population), reverse=True)]
    return selected[:len(population) // 2]  # Select top half

def genetic_algorithm_with_rhythm(scale_name):
    scale = scales[scale_name]
    population = [generate_melody_with_rhythm(scale) for _ in range(population_size)]  # Initialize population
    
    # Main GA loop
    for generation in range(generations):
        print(f"Generation {generation + 1}")

        # Evaluate fitness of the population
        fitness_values = [fitness_with_rhythm(melody, scale) for melody in population]
        best_fitness = max(fitness_values)
        print(f"Best fitness in this generation: {best_fitness}")

        # Selection: Select top half based on fitness
        selected_population = selection(population, fitness_values)

        # Crossover: Create new population
        new_population = []
        while len(new_population) < population_size:
            parent1, parent2 = random.sample(selected_population, 2)  # Pick two parents
            child1, child2 = crossover_with_rhythm(parent1, parent2)  # Create two children
            new_population.extend([child1, child2])

        # Mutation: Apply mutation to the new population
        population = [mutation_with_rhythm(melody, scale) for melody in new_population]

        # Print the best melody from the current generation
        best_melody = population[fitness_values.index(best_fitness)]
        print(f"Best melody: {best_melody}")

    # Return the best melody from the last generation
    fitness_values = [fitness_with_rhythm(melody, scale) for melody in population]
    best_melody = population[fitness_values.index(max(fitness_values))]
    return best_melody

# Run the genetic algorithm for C Major Scale with rhythms
best_melody = genetic_algorithm_with_rhythm('C_major')
print("Best melody after evolution:", best_melody)


Generation 1
Best fitness in this generation: 100
Best melody: [('E', 'eighth'), ('A', 'eighth'), ('C', 'whole'), ('E', 'eighth'), ('D', 'sixteenth'), ('D', 'sixteenth'), ('C', 'quarter'), ('B', 'eighth'), ('E', 'whole'), ('C', 'whole'), ('G', 'half'), ('F', 'quarter'), ('B', 'sixteenth'), ('B', 'whole'), ('A', 'half'), ('B', 'sixteenth'), ('B', 'whole'), ('E', 'quarter'), ('G', 'eighth'), ('D', 'quarter'), ('C', 'half'), ('D', 'eighth'), ('F', 'half'), ('E', 'sixteenth'), ('E', 'eighth'), ('G', 'quarter'), ('C', 'whole'), ('E', 'whole'), ('B', 'sixteenth'), ('D', 'whole'), ('A', 'whole'), ('C', 'sixteenth'), ('F', 'quarter'), ('B', 'quarter'), ('A', 'half'), ('D', 'whole'), ('G', 'whole'), ('A', 'whole'), ('A', 'whole'), ('E', 'eighth'), ('E', 'quarter'), ('A', 'sixteenth'), ('G', 'whole'), ('B', 'eighth'), ('E', 'half'), ('F', 'sixteenth'), ('G', 'quarter'), ('A', 'half'), ('C', 'quarter'), ('F', 'half'), ('C', 'half'), ('D', 'half'), ('G', 'eighth'), ('D', 'sixteenth'), ('F', 'sixte

In [11]:
import mido
from mido import MidiFile, MidiTrack, Message

# Map note names to MIDI numbers (C major scale for simplicity)
note_mapping = {
    'C': 60,  # Middle C
    'C#': 61, 
    'D': 62,
    'D#': 63,
    'E': 64,
    'F': 65,
    'F#': 66,
    'G': 67,
    'G#': 68,
    'A': 69,
    'A#': 70,
    'B': 71
}

# Rhythm durations in MIDI ticks
rhythm_durations = {
    'whole': 1920,    # 4 beats
    'half': 960,      # 2 beats
    'quarter': 480,   # 1 beat
    'eighth': 240,    # 0.5 beats
    'sixteenth': 120  # 0.25 beats
}

def create_midi(melody, filename):
    # Create a new midi file
    midi = MidiFile()
    track = MidiTrack()
    midi.tracks.append(track)

    # Add track name
    track.append(mido.MetaMessage('track_name', name='Melody'))

    # Set tempo (500000 microseconds per beat, which equals 120 BPM)
    track.append(mido.MetaMessage('set_tempo', tempo=500000))

    # Start time (initially set to 0)
    time = 0

    # Create the melody notes
    for note, rhythm in melody:
        # Convert note to MIDI number
        note_number = note_mapping[note]

        # Get the duration in ticks from the rhythm
        duration = rhythm_durations[rhythm]

        # Add a note_on message
        track.append(Message('note_on', note=note_number, velocity=64, time=time))

        # Add a note_off message after the note duration
        track.append(Message('note_off', note=note_number, velocity=64, time=duration))

        # Set time to zero for the next note to start immediately after the previous
        time = 0

    # Save the midi file to disk
    midi.save(filename)

# Example melody (the sequence you provided)
melody = [
    ('G', 'whole'), ('F', 'quarter'), ('G', 'sixteenth'), ('G', 'sixteenth'), 
    ('E', 'whole'), ('G', 'quarter'), ('E', 'eighth'), ('F', 'eighth'), 
    ('C', 'quarter'), ('F', 'sixteenth'), ('C', 'quarter'), ('E', 'quarter'), 
    ('F', 'sixteenth'), ('D', 'half'), ('G', 'eighth'), ('G', 'sixteenth')
    # Add more notes from your sequence here...
]

# Convert the melody to a MIDI file
create_midi(best_melody, 'ga_melody_complex.mid')


# Creating Music Sheet

In [12]:
import mido
from mido import MidiFile, MidiTrack, Message
from music21 import stream, note, metadata

# Map note names to MIDI numbers (C major scale for simplicity)
note_mapping = {
    'C': 60,  # Middle C
    'C#': 61, 
    'D': 62,
    'D#': 63,
    'E': 64,
    'F': 65,
    'F#': 66,
    'G': 67,
    'G#': 68,
    'A': 69,
    'A#': 70,
    'B': 71
}

# Rhythm durations in quarter notes (for music21)
rhythm_durations = {
    'whole': 4.0,    # 4 beats
    'half': 2.0,      # 2 beats
    'quarter': 1.0,   # 1 beat
    'eighth': 0.5,    # 0.5 beats
    'sixteenth': 0.25 # 0.25 beats
}

def create_music_sheet(melody):
    # Create a music21 Stream object (this will hold all the notes)
    melody_stream = stream.Stream()

    # Set metadata (optional)
    melody_stream.metadata = metadata.Metadata()
    melody_stream.metadata.title = "Generated Melody"
    melody_stream.metadata.composer = "AI Generated"

    # Add notes to the stream
    for note_name, rhythm in melody:
        # Convert note to music21 note object
        m21_note = note.Note(note_name, quarterLength=rhythm_durations[rhythm])
        melody_stream.append(m21_note)

    # Show the music sheet (it opens the music21 sheet view)
    # melody_stream.show()

    # Optionally, you can save the sheet music as a PDF or MusicXML
    # melody_stream.write('pdf', fp='generated_melody.pdf')  # Save as PDF
    melody_stream.write('musicxml', fp='generated_melody.xml')  # Save as MusicXML

# Example melody (the sequence you provided)
melody = [
    ('G', 'whole'), ('F', 'quarter'), ('G', 'sixteenth'), ('G', 'sixteenth'), 
    ('E', 'whole'), ('G', 'quarter'), ('E', 'eighth'), ('F', 'eighth'), 
    ('C', 'quarter'), ('F', 'sixteenth'), ('C', 'quarter'), ('E', 'quarter'), 
    ('F', 'sixteenth'), ('D', 'half'), ('G', 'eighth'), ('G', 'sixteenth')
    # Add more notes from your sequence here...
]

# Generate and display the music sheet
create_music_sheet(best_melody)

# Exporting to pdf

In [ ]:
#https://github.com/musescore/MuseScore/releases
#MuseScore-Studio-4.4.3.242971445.dmg

In [13]:
import subprocess
import os

# Path to your MusicXML file
musicxml_path = "generated_melody.xml"
pdf_output_path = "generated_melody.pdf"

# Path to MuseScore executable (modify according to your system)
musescore_executable = "/Applications/MuseScore 4.app/Contents/MacOS/mscore"

# Run the MuseScore command to convert MusicXML to PDF
subprocess.run([musescore_executable, musicxml_path, "-o", pdf_output_path])

print(f"PDF file generated: {pdf_output_path}")


qt.qml.typeregistration: Invalid QML element name "IconCode"; value type names should begin with a lowercase letter
qt.qml.typeregistration: Invalid QML element name "MusicalSymbolCodes"; value type names should begin with a lowercase letter
qt.qml.typeregistration: Invalid QML element name "ContainerType"; value type names should begin with a lowercase letter
qt.qml.typeregistration: Invalid QML element name "NavigationEvent"; value type names should begin with a lowercase letter
qt.qml.typeregistration: Invalid QML element name "MUAccessible"; value type names should begin with a lowercase letter
qt.qml.typeregistration: Invalid QML element name "CompareType"; value type names should begin with a lowercase letter
qt.qml.typeregistration: Invalid QML element name "SelectionMode"; value type names should begin with a lowercase letter
qt.qml.typeregistration: Invalid QML element name "ToolBarItemType"; value type names should begin with a lowercase letter
qt.qml.typeregistration: Invali

PDF file generated: generated_melody.pdf
